In [ ]:
# Validation split 방식과 KFold 방식에 차이
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Input
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [9]:
data = np.loadtxt('https://raw.githubusercontent.com/pykwon/python/refs/heads/master/testdata_utf8/prima-indians-diabetes.csv', 
                  delimiter=',', dtype=np.float32)

x = data[:, :-1]
y = data[:, -1]
# print(x[:3])
# print(y[:3])

def build_model():
    model = Sequential([
        Input(shape=(8,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid'),
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    return model

In [10]:
# Validation split
model_val = build_model()
history_val = model_val.fit(x,y, epochs=100, batch_size=8, validation_split=0.2, verbose=0)
val_acc = history_val.history['val_acc'][-1]

2025-09-15 16:24:16.519111: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [11]:
# KFold 방식 사용
kf = KFold(n_splits=5, shuffle=True, random_state=42)
kfold_accs = []

for train_idx, val_idx in kf.split(x):
    x_train, x_val = x[train_idx], x[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model_kf = build_model()
    model_kf.fit(x_train, y_train, epochs=100, batch_size=32, verbose=0)
    
    y_pred = model_kf.predict(x_val)
    y_pred_label = (y_pred >= 0.5).astype(int)
    acc = accuracy_score(y_val, y_pred_label)
    kfold_accs.append(acc)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/stepWARNING:tensorflow:5 out of the last 11 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x3472a2e80> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [12]:
# 비교 출력
print(f'[validation_split] 마지막 검증 정확도 : {val_acc:.4f}')
print(f'[KFold] 각 fold의 정확도 : {np.round(kfold_accs, 4)}')
print(f'[KFold] 평균 정확도 : {np.mean(kfold_accs):.4f}')

[validation_split] 마지막 검증 정확도 : 0.7078
[KFold] 각 fold의 정확도 : [0.6948 0.526  0.6364 0.6732 0.7124]
[KFold] 평균 정확도 : 0.6486
